In [1]:
import scipy.io
import scipy.ndimage
import numpy as np
from scipy.sparse.linalg import spsolve
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from Solver import nummodel, nummodel_jac, explicit_solve, implicit_solve
from NeuralNet import *
from timeit import default_timer



In [2]:
def permeability_ref(q, dq):
    return q**2 + dq**2 + 1
def D_permeability_ref(q, dq):
    return 2*q, 2*dq


# Training a quadratic function
 $$D(\theta) = \theta^2 + (\frac{\partial \theta}{\partial x})^2 + 1$$

In [ ]:
nn_save_name = "quadratic.nn"
N_train_1d = 100
L_train_1d = 1
N_train = N_train_1d**2

x_train_1d = np.linspace(-L_train_1d, L_train_1d, N_train_1d)
X_train_2d, Y_train_2d = np.meshgrid(x_train_1d, x_train_1d)


x_train = np.vstack((X_train_2d.reshape(-1), Y_train_2d.reshape(-1))).T
y_train = permeability_ref(x_train[:, 0], x_train[:, 1]).reshape((N_train, 1))
    
x_train = torch.from_numpy(x_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))

ind = x_train.shape[1]
outd = y_train.shape[1] 

learning_rate = 0.001
step_size = 100
gamma = 0.5  
epochs = 2000
batch_size = 64

layers = 2
width  = 20
nn_model = FNN(ind, outd, layers, width) 

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)

optimizer = Adam(nn_model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = torch.nn.MSELoss(reduction='sum')
t0 = default_timer()
for ep in range(epochs):
    nn_model.train()
    t1 = default_timer()
    train_l2 = 0
    for x, y in train_loader:

        optimizer.zero_grad()
        out = nn_model(x)
        
        loss = myloss(out , y)*100
        loss.backward()

        optimizer.step()
        train_l2 += loss.item()

    scheduler.step()

    if ep % 100 == 0:
        # train_l2/= ntrain
        t2 = default_timer()
        print("Epoch : ", ep, " Epoch time : ", t2-t1, " Train L2 Loss : ", train_l2)
    if nn_save_name is not None:
        torch.save(nn_model, nn_save_name)
        


Epoch :  0  Epoch time :  0.13650608000000375  Train L2 Loss :  6127754.883300781
Epoch :  100  Epoch time :  0.12998991500001011  Train L2 Loss :  486.16511952877045
Epoch :  200  Epoch time :  0.11405230799999799  Train L2 Loss :  317.2362446784973
Epoch :  300  Epoch time :  0.10966267899999593  Train L2 Loss :  307.9198333621025
Epoch :  400  Epoch time :  0.10767850799999223  Train L2 Loss :  304.23311130702496
Epoch :  500  Epoch time :  0.13186419300001262  Train L2 Loss :  303.38436844944954
Epoch :  600  Epoch time :  0.11113243599999123  Train L2 Loss :  302.2671687602997
Epoch :  700  Epoch time :  0.09765119799999411  Train L2 Loss :  301.9405007362366
Epoch :  800  Epoch time :  0.09891763699999956  Train L2 Loss :  301.9431594610214
Epoch :  900  Epoch time :  0.1027835720000212  Train L2 Loss :  301.717368721962
Epoch :  1000  Epoch time :  0.20939963699998998  Train L2 Loss :  301.67023861408234
Epoch :  1100  Epoch time :  0.11174314199996616  Train L2 Loss :  301.6388

# Testing on the quadratic function

In [ ]:
N_test_1d = 200
L_test_1d = 1
N_test = N_test_1d**2

x_test_1d = np.linspace(-L_test_1d, L_test_1d, N_test_1d)
X_test_2d, Y_test_2d = np.meshgrid(x_test_1d, x_test_1d)

x_test = np.vstack((X_test_2d.reshape(-1), Y_test_2d.reshape(-1))).T    
y_test = permeability_ref(x_test[:, 0], x_test[:, 1]).reshape((N_test, 1))



x_test = torch.from_numpy(x_test.astype(np.float32))
y_pred = nn_model(x_test).detach().numpy()



In [ ]:
y_test_2d = y_test.reshape((N_test_1d,N_test_1d))
y_pred_2d = y_pred.reshape((N_test_1d,N_test_1d))

plt.figure()
plt.pcolormesh(X_test_2d, Y_test_2d, y_test_2d)
plt.colorbar()

plt.figure()
plt.pcolormesh(X_test_2d, Y_test_2d, y_pred_2d)
plt.colorbar()

plt.figure()
plt.pcolormesh(X_test_2d, Y_test_2d, y_test_2d - y_pred_2d)
plt.colorbar()

# Derivative test

In [ ]:
q = np.array([1.0])
dq = np.array([1.0])
x = torch.from_numpy((np.vstack((q, dq)).T).astype(np.float32))  
x.requires_grad = True
y = nn_model(x)  #.detach().numpy().flatten()

d = torch.autograd.grad(y, x)

print("analytical derivative is :", d)
eps = 1e-3
xp0 = x + torch.Tensor([eps, 0.0])
yp0 = nn_model(xp0)
xp1 = x + torch.Tensor([0.0, eps])
yp1 = nn_model(xp1)

print("FD derivative is :", (yp0-y)/eps, (yp1-y)/eps)


# Plug-in test

In [ ]:
def permeability(q, dq):
    
    x = np.vstack((q, dq)).T
    
    permeability = nn_model(torch.from_numpy(x.astype(np.float32))).detach().numpy().flatten()
    
    return permeability

def D_permeability(q, dq):
    
    Ny = q.size
    Dq, Ddq = np.zeros(Ny), np.zeros(Ny)
    
    for i in range(Ny):
        x = torch.from_numpy(np.array([[q[i],dq[i]]]).astype(np.float32))
        x.requires_grad = True
        y = nn_model(x)  #.detach().numpy().flatten()
        d = torch.autograd.grad(y, x)[0].numpy().flatten()
        Dq[i], Ddq[i] = d[0], d[1]
    
    
    return Dq, Ddq

L = 1.0
Ny = 100
yy = np.linspace(0.0, L, Ny)
f = 6*(1-2*yy)**2 - 2*(yy - yy**2)*(1 - 2*yy)**2 + 2*(yy - yy**2)**2 + 2 
q_sol = -yy*(yy - 1)
dbc = np.array([0.0, 0.0]) 

MODEL = "imp_nummodel"

if MODEL == "imp_nummodel":
    
    model = lambda q, yy, res, V : nummodel_jac(permeability, q, yy, res, V, True, D_permeability)  
    yy, t_data, q_data = implicit_solve(model, f, dbc, dt = 1.0e-3, Nt = 200, save_every = 1, L = L)
    
    model = lambda q, yy, res, V : nummodel_jac(permeability_ref, q, yy, res, V, True, D_permeability_ref)  
    yy_ref, t_data_ref, q_data_ref = implicit_solve(model, f, dbc, dt = 1.0e-3, Nt = 200, save_every = 100, L = L)
    

elif MODEL == "imp_Newton_nummodel":
    
    model = lambda q, yy, res, V : nummodel_jac(permeability, q, yy, res, V, True, D_permeability)  
    yy, t_data, q_data = implicit_Newton_solve(model, f, dbc, dt = 1.0e-3, Nt = 200, save_every = 100, L = L, Newton_eps = 1e-3, Newton_maxiterstep = 1000)
    
    model = lambda q, yy, res, V : nummodel_jac(permeability_ref, q, yy, res, V, True, D_permeability_ref)  
    yy_ref, t_data_ref, q_data_ref = implicit_Newton_solve(model, f, dbc, dt = 1.0e-3, Nt = 200, save_every = 1, L = L, Newton_eps = 1e-3, Newton_maxiterstep = 1000)
     
else:
    print("ERROR")


plt.figure()
plt.plot(yy, q_data[-1, :],  "--o", fillstyle="none", label="q")
plt.plot(yy, q_data_ref[-1, :],  "--*", label="q ref")

plt.xlabel("y")
plt.legend()
plt.show()


 

